In [1]:
from pymongo import MongoClient
import requests
import json
import pandas as pd
from getpass import getpass

In [2]:
token = getpass()

········


### Conectar con la Collection en Mongo

In [3]:
def access_mdb_local_collection (database, collection):

    """ Function that returns a collection inside a DataBase 
    host in my local Server """

    # 1. Connect to my local Server
    client = MongoClient("localhost:27017")
    
    # 2. Access a DataBase:
    db = client[database]

    # 3. Acces a collection inside the given DataBase
    c = db.get_collection(collection)

    # 4. Return collection into a given DataBase host in a given Server
    return c

### Extrer datos de FourSquare:

In [4]:
def foursquare_places(venue,location,category = ""):
    
    """
    Search for UNLIMITED places in a 10km radius given a key_word
    query (venue), and returns a .json()
    
    """
    
    url = "https://api.foursquare.com/v3/places/search"

    params = {
        "query": venue,
        "ll": location,
        "radius":10000,
        "categories":category,
        "limit":50,
        #"sort":"DISTANCE" #elegir distancia o relevancia
    }

    headers = {
        "Accept": "application/json",
        "Authorization": token
    }

    return requests.get(url, params=params, headers=headers).json()


### Insertar los datos de FourSquare en la coleccion

In [5]:
def insert_parents_fourquare(response):
    
    """
    Inserts the results of a reques that ARE NOT in the 
    foursquare collection AND ARE NOT related to other object(childrens)
    """
    
    # 1. set conter for inserted objects
    inserted = 0
    
    # 2. access foursquare collection
    c = access_mdb_local_collection ("ironhack", "foursquare")
    
    # 3. Get the list of all FS elements alrready registered
    registered = [i['fsq_id'] for i in c.find({}, {"_id":0,"fsq_id":1})]
    
    # 4. Iterate trough all elements of the response 
    for i in response["results"]:
        
        #Need to meet 2 conditions to be inserted:
        # It is not alrready registered:
        cond1 = i['fsq_id'] not in registered
        
        ## It's not a children of a parent:
        cond2 = "parent" not in list(i["related_places"].keys())
        
        # Check if the conditions are meet:
        if cond1 and cond2 == True:
            c.insert_one(i)
            inserted += 1
            
    print(f'{inserted} items inserted into foursquare collection')
               
    return

# MAIN:

In [10]:
locations = {
    "Valencia" : "39.47534441,-0.37565717"
}

"""
          
    - API FourSquare: 
    
           - Airpot or Trainstation
           - Starbuks near 
            - Nigth clubs

19031	Travel and Transportation > Transport Hub > Airport
19042	Travel and Transportation > Transport Hub > Bus Station
19066	Travel and Transportation > Train
19000	Travel and Transportation

13035	Dining and Drinking > Cafe, Coffee, and Tea House > Coffee Shop

13003	Dining and Drinking > Bar
10032	Arts and Entertainment > Night Club

"""
            
categories = {
    "transport" : 19000,
    "coffee" : 13035,
    "nigth_club" : 13003,
}



for key, loc in locations.items():
    for key2, cat in categories.items():
        
        venue = ""
        location = loc
        category = cat
        
        print(f'----Location {key}, category {key2}----')
        
        response = foursquare_places(venue,location,category)
    
        insert_parents_fourquare(response)
              
        print(f'---------------------------------------')
        

----Location Valencia, category transport----
0 items inserted into foursquare collection
---------------------------------------
----Location Valencia, category coffee----
0 items inserted into foursquare collection
---------------------------------------
----Location Valencia, category nigth_club----
43 items inserted into foursquare collection
---------------------------------------


In [ ]:
# Salen tantos 0 porque ya habiamos guardado antes datos de Madrid y Valencia